In [1]:
import pandas as pd
from docplex.mp.model import Model
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import QuadraticProgramToQubo
import numpy as np
import dimod
from dwave.system import DWaveSampler
from dwave_qbsolv import QBSolv

In [2]:

import neal
from dwave.system import LeapHybridSampler

In [3]:
numLocs = 80
numDCs = 5
capDC = 40
maxDCs = 2
DCs = ["Thane", "Powai", "CST", "Vashi", "Panvel"]

locsServed = {"Thane": 10, 
              "Powai": 20, 
              "CST": 30, 
              "Vashi": 10, 
              "Panvel": 10}



distMat = pd.DataFrame({"Thane": [0, 5, 20, 8, 25],
                        "Powai": [5, 0, 15, 12, 27],
                        "CST": [20, 15, 0, 10, 30],
                        "Vashi": [8, 12, 10, 0, 10],
                        "Panvel": [25, 27, 30, 10, 0]})


In [4]:
mdl = Model(name='Toy Problem - 5 DCs')

x = mdl.binary_var_dict([(i, j) for j in range(numDCs) for i in range(numDCs)], name='x')

obj = mdl.sum(distMat.iloc[i, j] * x[i, j] for j in range(numDCs) for i in range(numDCs))
mdl.minimize(obj)

con1 = mdl.add_constraints(mdl.sum(x[i, j] for j in range(numDCs)) == 1 for i in range(numDCs))

con2 = mdl.add_constraints(mdl.sum(x[i, j] * list(locsServed.values())[i] for i in range(numDCs)) 
                           <= capDC * x[j, j] for j in range(numDCs))     

con3 = mdl.add_constraint(mdl.sum(x[j, j] for j in range(numDCs)) == maxDCs)

In [5]:
qp = from_docplex_mp(mdl)
qubo = QuadraticProgramToQubo().convert(qp)

In [6]:
bqm_binary = dimod.as_bqm(qubo.objective.linear.to_array(), qubo.objective.quadratic.to_array(), dimod.BINARY)

In [9]:
import dimod
import hybrid

In [ ]:
# Define the workflow
iteration = hybrid.RacingBranches(
    hybrid.InterruptableTabuSampler(),
    hybrid.EnergyImpactDecomposer(size=2)
    | hybrid.QPUSubproblemAutoEmbeddingSampler()
    | hybrid.SplatComposer()
) | hybrid.ArgMin()
workflow = hybrid.LoopUntilNoImprovement(iteration, convergence=3)

In [ ]:
init_state = hybrid.State.from_problem(bqm_binary)

In [7]:
sampler = QBSolv()

result = sampler.sample(bqm_binary, label="example_qp",num_reads=1000)

In [8]:
print(result.first)

Sample(sample={0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 1, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 1, 19: 1, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 1, 31: 0, 32: 1, 33: 0, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 0, 41: 1, 42: 0, 43: 1, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0}, energy=-2585.0, num_occurrences=1)
